## Deliverable 2. Create a Customer Travel Destinations Map.

In [58]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [59]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Saint-Philippe,RE,-21.36,55.77,73.40,78,0,6.93,clear sky
1,1,Norman Wells,CA,65.28,-126.83,35.60,74,75,8.05,broken clouds
2,2,Rosarito,MX,32.33,-117.03,80.60,28,20,10.29,few clouds
3,3,Buala,SB,-8.14,159.59,85.21,67,100,5.99,light rain
4,4,Cape Town,ZA,-33.93,18.42,61.00,59,0,21.92,clear sky
...,...,...,...,...,...,...,...,...,...,...
698,698,Zonguldak,TR,41.25,31.83,39.96,89,8,2.35,clear sky
699,699,Okhotsk,RU,59.38,143.30,-7.31,86,63,15.23,broken clouds
700,700,Barcelos,PT,41.54,-8.62,51.01,76,0,3.36,clear sky
701,701,Batie,BF,9.88,-2.92,77.02,21,0,2.10,clear sky


In [60]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [61]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Rosarito,MX,32.33,-117.03,80.60,28,20,10.29,few clouds
3,3,Buala,SB,-8.14,159.59,85.21,67,100,5.99,light rain
9,9,Cabo San Lucas,MX,22.89,-109.91,78.01,57,5,3.36,clear sky
10,10,Kapaa,US,22.08,-159.32,80.60,65,1,6.93,clear sky
11,11,Samarai,PG,-10.62,150.67,82.02,80,99,12.50,light rain
...,...,...,...,...,...,...,...,...,...,...
682,682,Maragogi,BR,-9.01,-35.22,78.15,82,19,8.52,few clouds
688,688,Kroya,ID,-7.63,109.25,78.30,82,92,3.74,overcast clouds
693,693,Itaituba,BR,-4.28,-55.98,81.30,72,81,6.08,light rain
701,701,Batie,BF,9.88,-2.92,77.02,21,0,2.10,clear sky


In [62]:
# 4a. Determine if there are any empty rows.
if preferred_cities_df.isnull().values.any() == True:
    print("Null values present in the Preferred cities list")

In [63]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
if preferred_cities_df.isnull().values.any() == True:
    preferred_cities_df = preferred_cities_df.dropna()
    print("Null values present in the Preferred cities list so dropped na rows")
preferred_cities_clean_df = preferred_cities_df
print("no nulls in preferred cities list")

no nulls in preferred cities list


In [64]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
# Populate None as the value under "Hotel Name" column initially
hotel_df["Hotel Name"] = None
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Rosarito,MX,80.60,few clouds,32.33,-117.03,None
3,Buala,SB,85.21,light rain,-8.14,159.59,None
9,Cabo San Lucas,MX,78.01,clear sky,22.89,-109.91,None
10,Kapaa,US,80.60,clear sky,22.08,-159.32,None
11,Samarai,PG,82.02,light rain,-10.62,150.67,None
...,...,...,...,...,...,...,...
682,Maragogi,BR,78.15,few clouds,-9.01,-35.22,None
688,Kroya,ID,78.30,overcast clouds,-7.63,109.25,None
693,Itaituba,BR,81.30,light rain,-4.28,-55.98,None
701,Batie,BF,77.02,clear sky,9.88,-2.92,None


In [65]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print(f"Hotel not found... skipping.{index}")
        

Hotel not found... skipping.145
Hotel not found... skipping.161
Hotel not found... skipping.225
Hotel not found... skipping.242
Hotel not found... skipping.256
Hotel not found... skipping.281
Hotel not found... skipping.309
Hotel not found... skipping.316
Hotel not found... skipping.349
Hotel not found... skipping.389
Hotel not found... skipping.401
Hotel not found... skipping.404
Hotel not found... skipping.420
Hotel not found... skipping.426
Hotel not found... skipping.446
Hotel not found... skipping.463
Hotel not found... skipping.615


In [68]:
# 7. Drop the rows where there is no Hotel Name.
hotel_clean_df = hotel_df.dropna()
hotel_clean_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Rosarito,MX,80.60,few clouds,32.33,-117.03,Hotel Festival Plaza Rosarito
3,Buala,SB,85.21,light rain,-8.14,159.59,Maringe Lagoon Lodge
9,Cabo San Lucas,MX,78.01,clear sky,22.89,-109.91,Hotel Tesoro Los Cabos
10,Kapaa,US,80.60,clear sky,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
11,Samarai,PG,82.02,light rain,-10.62,150.67,Nuli Sapi
...,...,...,...,...,...,...,...
682,Maragogi,BR,78.15,few clouds,-9.01,-35.22,Salinas do Maragogi All Inclusive Resort
688,Kroya,ID,78.30,overcast clouds,-7.63,109.25,RUSTANTO Fam
693,Itaituba,BR,81.30,light rain,-4.28,-55.98,Hotel Campos
701,Batie,BF,77.02,clear sky,9.88,-2.92,Auberge SONDAR AFACI


In [69]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_clean_df.to_csv(output_data_file, index_label="City_ID")

In [72]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_clean_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_clean_df[["Lat", "Lng"]]

In [73]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_clean_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:
# ********* Create a Customer Travel Destinations Map complete ***********